<a href="https://colab.research.google.com/github/nasa/PeTaL-labeller/blob/JQ/auto-labeler/auto_labeler_prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install tensorboardX
!pip install wikipedia
!pip install swifter

In [ ]:
import torch
import tensorflow as tf
import pandas as pd
import wikipedia 
import swifter
import numpy as np

## GPU Detection

In [ ]:
# GPU detection 

# Get GPU device name
device_name = tf.test.gpu_device_name()

if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
# If there is a GPU available
if torch.cuda.is_available():    

    # Tell PyTorch to use GPU
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


## Import, Parse, and Store Data

In [ ]:
#Creating PyDrive instance to load in data from PeTaL shared drive, follow the steps to authenticate
!pip install -U -q PyDrive 
  
from pydrive.auth import GoogleAuth 
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials 
  
  
# Authenticate and create the PyDrive client. 
auth.authenticate_user() 
gauth = GoogleAuth() 
gauth.credentials = GoogleCredentials.get_application_default() 
drive = GoogleDrive(gauth)

In [ ]:


#this is the un-parsed articles
# link = 'https://drive.google.com/file/d/1iIZgKs1swHHJuumCU5xyW8tXSAnKAg18/view?usp=sharing'
# id = link.split("/")[-2] 
  
# downloaded = drive.CreateFile({'id':id})  
# downloaded.GetContentFile('articles.csv')   
#df = pd.read_csv('articles.csv')


In [ ]:
#'https://petscan.wmflabs.org/' link to pull wikipedia articles and their page ID's

In [ ]:
#Scraping article content by ID
def wiki_content(row):
  id = row['pageid']
  try:
    content = wikipedia.page(pageid=id).content
  except:
    content = 'error'
  return content

df['Content'] = df.swifter.apply(wiki_content, axis=1)

In [ ]:
#Scraping article summary by ID

def wiki_summary(row):
  id = row['pageid']
  try:
    summary = wikipedia.page(pageid=id).summary
  except:
    summary = 'error'
  return summary

df['Summary'] = df.swifter.apply(wiki_summary, axis=1)

In [ ]:
#Saving parsed articles as csv, can be accessed in the "Files" folder on the left, then download if you want
df.to_csv('parsed_articles.csv')

In [ ]:
#Google drive link to the parsed articles
link = 'https://drive.google.com/file/d/1XRWsEsNUHjWOjPavwrfuUpaq3DwGGE4D/view?usp=sharing'
id = link.split("/")[-2] 
 
downloaded = drive.CreateFile({'id':id})  
downloaded.GetContentFile('parsed_articles.csv') 
df = pd.read_csv('parsed_articles.csv')

df = df[(df['Content'] != 'error') & df['Content'].notnull()]

#Df 'Content' column into list
docs = list(df['Content'].values)

In [ ]:
#Labels

labels = ['Maintain homeostasis', 'Protect from temperature']

In [ ]:
df['Content'].value_counts().to_frame()

## Helper Functions

In [ ]:
# Calculate accuracy of predictions vs labels
import numpy as np

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
# Format elapsed times as hh:mm:ss
import time
import datetime

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

## BERT Tokenizer

In [ ]:
from transformers import BertModel, BertConfig

# Load BERT tokenizer
print('Loading BERT tokenizer')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer


In [ ]:
# Make sure it is tokenizing correctly:

# Print original articles
print(' Original: ', docs[0])

# Print a doc split into tokens
print('Tokenized: ', tokenizer.tokenize(docs[0]))

# Print docs as mapped to ids
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(docs[0])))

In [ ]:
max_len = 0

for d in docs:

    # Tokenize text and add `[CLS]` and `[SEP]` tokens
    input_ids = tokenizer.encode(d, add_special_tokens=True)

    # Update max length
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

In [ ]:
# Finishing tokenizing all docs and map tokens to thier word IDs
input_ids = []
attention_masks = []

for d in docs:

    encoded_dict = tokenizer.encode_plus(
                        d,                      # Docs to encode.
                        truncation=True,
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 256,           # Pad & truncate all docs
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Attention masks
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    input_ids.append(encoded_dict['input_ids'])
    
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)


labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', docs[0])
print('Token IDs:', input_ids[0])
print('Reverse:', tokenizer.convert_ids_to_tokens(input_ids[0]))

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


ValueError: ignored

In [ ]:
# Split up training & testing/validation

from torch.utils.data import TensorDataset, random_split

dataset = TensorDataset(input_ids, attention_masks, labels)

# 80:20 split

# Number of docs to include per set
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training docs'.format(train_size))
print('{:>5,} validation docs'.format(val_size))

In [ ]:
# Iterator using torch DataLoader class so that entire dataset doesn't need to be stored in memory

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# batch size can be 16 or 32
batch_size = 32

# Sample in random order when training
train_dataloader = DataLoader(
            train_dataset,  
            sampler = RandomSampler(train_dataset), 
            batch_size = batch_size 
        )

# Sample sequentially for validation
validation_dataloader = DataLoader(
            val_dataset, 
            sampler = SequentialSampler(val_dataset), 
            batch_size = batch_size # Evaluate with this batch size.
        )

## Training the Classification Model w/ Sequence Classification
  (fine-tune BERT)

  [HuggingFace documentation](https://huggingface.co/transformers/v2.2.0/model_doc/bert.html)

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# BertForSequenceClassification -> BERT model w/ added classification layer 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # 12-layer model, uncased vocab
    num_labels = 7, # Number of labels 
    """CHANGE ABOVE"""
    output_attentions = False, 
    output_hidden_states = False, 
)

# this needs to be run on GPU
model.cuda()

## Optimizer for our hypermarameters / Learning Rate Scheduler
AdamW

Possible hyperparamters: 
* batch size: 16, 32
* learning rate: 5e-5, 3e-5, 2e-5
* number of epochs: 2, 3, 4

In [ ]:
# Exeprimenting w/ different parameters
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, 
                  eps = 1e-8 # epsilon prevents division by 0??
                )

In [ ]:
from transformers import get_linear_schedule_with_warmup

# Training epochs should be betw 2- 4 (reduce if overfitting)
epochs = 4

total_steps = len(train_dataloader) * epochs

# LR scheduler
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, 
                                            num_training_steps = total_steps)

## Training Loop

In [ ]:
import random

# based on huggingface transformers `run_glue.py` script : https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

training_vals = []
total_time = time.time()

for epoch_i in range(0, epochs):       
    ## TRAINING
    print("")
    print('-------- Epoch {:} / {:} --------'.format(epoch_i + 1, epochs))
    print('Training...')
    print("")

    t0 = time.time()
    total_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):
        if step % 20 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            # progress from every 20 batches
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # pytorch tensors in batch (gpu usage)
        #   batch[0] -> input ids 
        #   batch[1] -> attention masks
        #   batch[2] -> labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # clear gradients before forward and backward passes
        model.zero_grad()        

        # forward pass
        loss, logits = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
        total_loss += loss.item()

        # backward pass
        loss.backward()

        # clip to prevent exploding gradients (??)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # optimizer and lr updare
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader)            
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    ## VALIDATION
    
    print("")
    print("Validation...")

    t0 = time.time()
    model.eval()

    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # pytorch tensors in batch (gpu usage)
        #   batch[0] -> input ids 
        #   batch[1] -> attention masks
        #   batch[2] -> labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        with torch.no_grad():        

            # forward pass
            (loss, logits) = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
        total_eval_loss += loss.item()

        # move logits and labels -> CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()  
        total_eval_accuracy += flat_accuracy(logits, label_ids)

    # print final validation accuracy
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    avg_val_loss = total_eval_loss / len(validation_dataloader)   
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # epoch values & stats
    training_vals.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )
print("")
print("Training done")
print("Time to train: {:} (h:mm:ss)".format(format_time(time.time()-total_time)))

In [ ]:
# Display metrics of training process in a dataframe

import pandas as pd

pd.set_option('precision', 2)

df_vals = pd.DataFrame(data=training_stats)
df_vals = df_vals.set_index('epoch')

df_vals

In [ ]:
input_ids_test = []
attention_masks_test = []
actual_labels_test=[]

for i in range(500):

    encoded_dict = tokenizer.encode_plus(
                        sentences_test[i],                      
                        add_special_tokens = True, 
                        max_length = 256,           
                        pad_to_max_length = True,
                        return_attention_mask = True,   
                        return_tensors = 'pt',     
                   )
    
   
    input_ids_test.append(encoded_dict['input_ids'])
    

    attention_masks_test.append(encoded_dict['attention_mask'])
    actual_labels_test.append(labels_test[i])

# lists -> tensors
input_ids_test = torch.cat(input_ids_test, dim=0)
attention_masks_test = torch.cat(attention_masks_test, dim=0)
actual_labels_test = torch.tensor(actual_labels_test)

batch_size = 32  

# build DataLoader
prediction_data = TensorDataset(input_ids_test, attention_masks_test, actual_labels_test)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

## Testing Classification

In [ ]:
##

In [ ]:
## Ignore this cell for now
# Trying out example BERT

# Single training/test example for simple sequence classification
class InputExample(object):

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.

        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label


class InputFeatures(object):
    """Single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_id):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id

In [ ]:
# Initializing a BERT bert-base-uncased style configuration
configuration = BertConfig()

# Initializing a model from the bert-base-uncased style configuration
model = BertModel(configuration)

model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)

In [ ]:
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)